In [ ]:
!pip install pygad

In [ ]:
import pandas as pd
import pygad
import numpy 

In [ ]:
data=pd.read_csv('/content/npc 18 test for colab 5  for ga.csv')

In [ ]:
from sklearn.model_selection import train_test_split
X=data.iloc[:,:4].values
Y = data.iloc[:,4].values

train_data,test_data,train_targets,test_targets=train_test_split(X,Y,test_size=0.1,random_state=42)
train_data,predict_data,train_targets,predict_targets=train_test_split(train_data,train_targets,test_size=0.1,random_state=42)

In [ ]:
print(test_data[0])
print(test_targets[0])
print(predict_data[0])

ANN modelling using boston method


In [ ]:
#mean = train_data.mean(axis=0)
#train_data -= mean
#std = train_data.std(axis=0)
#train_data /= std
#test_data -= mean
#test_data /= std

In [ ]:
#build network

from keras import models
from keras import layers

def build_model():
  model=models.Sequential()
  model.add(layers.Dense(6,activation='relu',input_shape=(train_data.shape[1],)))
  #model.add(layers.Dense(14, kernel_initializer='normal', activation='relu'))
  model.add(layers.Dense(6, kernel_initializer='normal', activation='relu'))
  model.add(layers.Dense(1)),#activation='relu')
  model.compile(optimizer='adam', loss='mse', metrics=['mae'])
  return model

Pre-test

In [ ]:
import numpy as np

k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]
all_scores2=[]

for i in range(k):
  print('processing fold#',i)
  val_data=train_data[i*num_val_samples:(i+1)*num_val_samples]
  val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)

  model=build_model()
  model.fit(partial_train_data,partial_train_targets,epochs=num_epochs,verbose=0,batch_size=1)
  val_mse,val_mae=model.evaluate(val_data,val_targets,verbose=0)
  all_scores.append(val_mae)
  all_scores2.append(val_mse)
 
  

In [ ]:
print(all_scores)
print(all_scores2)
print(np.mean(all_scores))
print(np.mean(all_scores2))

large epoch test

In [ ]:
num_epochs = 2500
all_mae_histories = []
all_mse_histories = []
for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
  model = build_model()
  history = model.fit(partial_train_data, partial_train_targets,validation_data=(val_data, val_targets),epochs=num_epochs, batch_size=1, verbose=0)
  mae_history = history.history['val_mae']
  
  all_mae_histories.append(mae_history)


In [ ]:
average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]


MAE-epochs graphing

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, len(average_mae_history) + 1), average_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()


In [ ]:
def smooth_curve(points, factor=0.9):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor)) 
    else:
      smoothed_points.append(point)
  return smoothed_points
smooth_mae_history = smooth_curve(average_mae_history[10:])
plt.plot(range(1, len(smooth_mae_history) + 1), smooth_mae_history)
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()

In [ ]:
model = build_model()
model.fit(train_data, train_targets,epochs= 2000, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)
print("Test MAE:",test_mae_score)
print("Test MSE:",test_mse_score)
  

In [ ]:
prediction=model.predict(predict_data)
print(prediction)
print(predict_targets)

#Defining MAPE function
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

LR_MAPE= MAPE(predict_targets,prediction)
print("MAPE: ",LR_MAPE)
pred_mse_score, pred_mae_score = model.evaluate(predict_data, predict_targets)
print("Pred MAE:",pred_mae_score)
print("Pred MSE:",pred_mse_score)
  

GA part

In [ ]:
desired_output=45

In [ ]:
def fitness_func(solution,solution_idx):  
    solution_fitness = 1.0/numpy.abs(pred_mse_score)
    return solution_fitness

In [ ]:
num_generations = 100 # Number of generations.
num_parents_mating = 10 # Number of solutions to be selected as parents in the mating pool.

sol_per_pop = 20 # Number of solutions in the population.
num_genes=11

last_fitness = 1
def on_generation(ga_instance):
    global last_fitness
    print("Generation = {generation}".format(generation=ga_instance.generations_completed))
    print("Fitness    = {fitness}".format(fitness=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]))
    print("Change     = {change}".format(change=ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1] - last_fitness))
    last_fitness = ga_instance.best_solution(pop_fitness=ga_instance.last_generation_fitness)[1]

ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       sol_per_pop=sol_per_pop,
                      fitness_func=fitness_func,
                       num_genes=num_genes,
                       on_generation=on_generation)

# Running the GA to optimize the parameters of the function.
ga_instance.run()

ga_instance.plot_fitness()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution(ga_instance.last_generation_fitness)
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Index of the best solution : {solution_idx}".format(solution_idx=solution_idx))

predictionga =model.predict(solution) 
print("Predicted output based on the best solution : {predictionga}".format(predictionga=predictionga))

if ga_instance.best_solution_generation != -1:
    print("Best fitness value reached after {best_solution_generation} generations.".format(best_solution_generation=ga_instance.best_solution_generation))